In [4]:
# relevant packages & modules
import re
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
import time
import requests
import urllib.request
import json
from tqdm import tqdm
# from tqdm.notebook import tqdm
import undetected_chromedriver.v2 as uc
from IPython.display import clear_output
import pandas as pd


# Have powerfull bot protection, so we need undetected_chrome
class ScrapperPlusvalia:
    
    driver = None

    
    logged_in = False
    
    def __init__(self):
        
        self.driver = uc.Chrome(version_main=95)
        self.root = "https://www.plusvalia.com/inmuebles-ordenado-por-fechaonline-descendente"
        self.base = f"{self.root}-pagina-"
        self.driver.get(f"{self.base}1.html")

    def wait(self, t):
        time.sleep(t)
        
    def check_requisites(self):     
        return True
    
    def collect_urls(self,page_from,page_to,waiting_time=2)->list:
        
        if not self.check_requisites():
            return []

        urls = []
        repeated = []
        

        with open("urls.txt","a") as f:

            for page_i in tqdm(range(page_from, page_to+1)):

                url = f"{self.base}{page_i}.html"
                self.driver.get(url)
                
                chunk = self.driver.find_elements(By.XPATH, '//div[contains(@class,"list-card-container")]/div')
                
                for c in chunk:
                    if str(c.get_attribute("class")) != "ads-container":

                        url_item = str(c.get_attribute("data-to-posting"))
                        if url_item is None:
                            pass
                        else:
                            if url_item not in urls:
                                urls.append(url_item)
                            else:
                                repeated.append(f"Repeated inmuebles-ordenado-por-fechaonline-descendente-{page_i}.html:{url_item}")

                if len(urls) > 0:
                    f.write("\n".join(map(lambda s: f"https://www.plusvalia.com{s}", urls)))
                    urls = []

                current_url = self.driver.current_url


                if current_url == f"{self.root}.html":
                    page = 1
                else:
                    page = int(re.match(r"^https\:\/\/www\.plusvalia\.com\/[A-z\-]*(\d+)\.html$", current_url).groups()[0])

                if page == page_to:
                    break
                
                # next = self.driver.find_elements(By.CSS_SELECTOR, 'li.pag-go-next > a')[0]
                # next.click()
                self.wait(waiting_time)

        return urls, repeated


    def scrape_items(self,urls_items)->list:
        
        if not self.check_requisites():
            return []

        
    
    def close(self):
        if self.driver:
            self.driver.close()
            
    def __del__(self):
        print("Closing...")
        self.close()

/home/magody/anaconda3/lib/python3.8/site-packages/requests/__init__.py:89: RequestsDependencyWarning: urllib3 (1.26.8) or chardet (3.0.4) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "


In [5]:
scrapper = ScrapperPlusvalia()
# urls, _ = scrapper.collect_urls(1,3468,1.5)  # 3468

In [3]:
import pandas as pd
import numpy as np
with open("urls.txt") as f:
    lines = f.readlines()
    print(len(lines))
    lines = np.unique(lines)
    print(len(lines))

    with open("urls_clean.txt","w") as f_out:
        f_out.writelines(map(lambda s: s, lines))

36785
17750


In [8]:


def extract_first(elements, mapping=None, default="none"):
    elements_len = len(elements)
    if elements_len == 0:
        return default
    
    
    try:
        element = elements[0].text
        if mapping is not None:
            element = mapping(element)
        return element
    except:
        return default


def parse_multiple(web_elements, separator="|"):
    
    out = ""
    for i in range(len(web_elements)):
        out += web_elements[i].text
        if i < len(web_elements)-1:
            out += separator        
    return out
    
class ItemRealState:
    
    attributes = []
    
    def set_attributes(self, dict_attributes:dict):
        self.attributes = []
        for key,value in dict_attributes.items():
            setattr(self, key, value)
            self.attributes.append(key)
    
    def get_serie(self, map_attributes_to_columns=dict()):
        if len(self.attributes) == 0:
            print("First add atributes with set_attributes")
            return pd.Series()
        
        if len(map_attributes_to_columns) > 0:
            # not supported
            return pd.Series()
        
        data = dict()
        for attribute in self.attributes:
            data[attribute] = getattr(self, attribute)
        
        return pd.Series(data=data)
            
"""
### REFERENCES
https://www.plusvalia.com/propiedades/vendo-terreno-con-casa-valle-de-los-chillos-sector-60551153.html
https://www.plusvalia.com/propiedades/venta-en-remate-de-hermosa-casa-en-ibarra-ahora-63081714.html
https://www.plusvalia.com/propiedades/alquiler-departamento-sector-gonzalez-suarez-60929588.html
https://www.plusvalia.com/casas-en-venta-ordenado-por-precioxm2-descendente.html#
https://www.plusvalia.com/propiedades/vendo-o-arriendo-casa-amoblada-urbanizacion-cerrada-63007741.html
https://www.plusvalia.com/casas-en-venta-ordenado-por-cantidadvisitas-descendente.html#
https://www.plusvalia.com/propiedades/zaigen-oficinas-y-locales.-sector-plataforma-55554056.html?labs=developments&userid=null&itemid=62559412&labs_source=RECOMENDADOS_FICHA_PROPIEDAD_DESKTOP


self.cache["urls"] = urls_items
self.cache["index_seen"] = -1
self.cache["urls_error"] = []
self.cache["df"] = df

for line in ["https://www.plusvalia.com/propiedades/casa-calderon-63208500.html"]:
# line = "https://www.plusvalia.com/propiedades/casa-calderon-63208500.html"
scrapper.driver.get(line)
scrapper.wait(1)
"""
import datetime
##
restarting = False
cache = dict()
stop = False
##
today = datetime.date.today()

def dateToString(date):
    return f"{date.day}/{date.month}/{date.year}"
today_str = dateToString(today)

web_elements_location_tree = scrapper.driver.find_elements(By.XPATH, "//ul[@class='breadcrumb']/li")

location_tree = parse_multiple(web_elements_location_tree,"->")

type_real_state = web_elements_location_tree[1].text.strip().lower()

if type_real_state == "casa":
    title = extract_first(scrapper.driver.find_elements(By.XPATH, "//div[@class='section-title']"))
    location_string = extract_first(scrapper.driver.find_elements(By.XPATH, "//h2[@class='title-location']")).replace("\nVer en mapa", "")
    
    url_maps = scrapper.driver.find_elements(By.XPATH, "//img[@id='static-map']")[0].get_attribute("src")
    location_latitude, location_longitude = re.match(r".*markers=(\-?\d+\.[\d]+),(\-?\d+\.[\d]+).*",url_maps).groups()
    
    date_publication = extract_first(scrapper.driver.find_elements(By.XPATH, "//div[@id='user-views']/div/div[1]"))
    views = int(extract_first(scrapper.driver.find_elements(By.XPATH, "//div[@id='user-views']/div/div[2]"),default="0 visualizaciones").split(" ")[0])
    if date_publication == "Publicado hoy":
        date_publication = today_str
    else:
        days = int(re.match("Publicado hace (\d+).*", date_publication).groups()[0])
        date_publication = dateToString(today - datetime.timedelta(days=days))

    state = extract_first(scrapper.driver.find_elements(By.XPATH, "//div[@class='price-operation']"))
    deliver = ""
    price_items_split = extract_first(scrapper.driver.find_elements(By.XPATH, "//div[@class='price-items']/span/span")).split("\n")
    coin, price = re.match("([A-z]+) ([\d\.]+)", price_items_split[0]).groups()
    price = float(price.replace(".",""))
    discount = 0 if len(price_items_split) == 1 else int(price_items_split[1].replace("%",""))
    
    description_sup = extract_first(scrapper.driver.find_elements(By.XPATH, "//h2[@class='title-type-sup']"))
    description = extract_first(scrapper.driver.find_elements(By.XPATH, "//div[@id='longDescription']"))
    
    web_elements_imgs = scrapper.driver.find_elements(By.XPATH, "//div[@id='react-gallery']//img")
    pictures = ""
    for i,element in enumerate(web_elements_imgs):
        pictures += element.get_attribute("src")
        if i < len(web_elements_imgs)-1:
            pictures += "|"
    
    advertiser_name = extract_first(scrapper.driver.find_elements(By.XPATH, "//section[@id='reactPublisherData']/div/div"))
    advertiser_codes = parse_multiple(scrapper.driver.find_elements(By.XPATH, "//section[@id='reactPublisherData']/div/ul/li"))

    feature_unities = ""
    feature_stotal = ""
    feature_scover = ""
    feature_bath = ""
    feature_parking = ""
    feature_toilete= ""
    feature_antique= ""


    extra_discover = ""
else:
    title = ""
    location_string = ""
    location_latlon = ""
    date_publication = ""
    state = ""
    deliver = ""
    price = ""
    description_sup = ""
    description = ""
    pictures = ""
    advertiser_code = ""
    advertiser_code_plusvalia = ""
    advertiser_lic= ""

    feature_unities = ""
    feature_stotal = ""
    feature_scover = ""
    feature_bath = ""
    feature_parking = ""
    feature_toilete= ""
    feature_antique= ""


    extra_discover = ""


# "location_tree":location_tree, include
attributes = {}







'USD 67.000\n39%'

In [57]:
parse_multiple(scrapper.driver.find_elements(By.XPATH, "//section[@id='reactPublisherData']/div/ul/li"))




'Cód. del anunciante: 2EMJ1N|Cód. Plusvalia: 60551153'

In [38]:
today  - datetime.timedelta(days=10)

datetime.date(2022, 1, 21)